The notebook has the following sections:
1. Import full dataset and keep only big cluster
2. Model helpers and parameters
3. Model definition
4. Model training
5. Prediction
6. Evaluate on test data

In [1]:
import pandas as pd
import numpy as np
import os
import pickle

print(os.path.abspath('.'))

/Users/d071503/Desktop/Uni/projects/datamining2


# 1. Import full dataset and keep only big cluster

## If you already have the X_big and Y_big dataset, skip this part and go to 'Load big cluster'

In [2]:
# Import cluster identifier
sales = pd.read_csv('./data/clean/data_v0.1_sales.csv')
big_key = sales['key'][sales['cluster'] == "big"]
print(len(big_key.unique())) # Should only have 2907 keys remaining
print(big_key.head())

2907
2      100033 (35-38 )
38              10063L
39              10063M
40             10063XL
48    100652 ( 37-39 )
Name: key, dtype: object


In [3]:
# Import datasets
X_full = pickle.load(open('./data/clean/X_flat.pkl', 'rb'))
Y_full = pickle.load(open('./data/clean/Y_flat.pkl', 'rb'))

print(X_full.shape)
print(Y_full.shape)
print(X_full.columns)
print(Y_full.columns)

(1577352, 108)
(1577352, 3)
Index(['key', 'pid_x', 'size_x', 'color', 'brand', 'rrp', 'mainCategory',
       'category', 'subCategory', 'releaseDate',
       ...
       'cat_7', 'cat_10', 'cat_16', 'cat_18', 'cat_24', 'cat_30', 'cat_33',
       'cat_36', 'cat_37', 'marketing_activity'],
      dtype='object', length=108)
Index(['key', 'date', 'sales'], dtype='object')


In [4]:
# Dump to pickle for faster loading of data in future
pickle.dump(X_big, open('./data/clean/X_big.pkl', 'wb'))
pickle.dump(Y_big, open('./data/clean/Y_big.pkl', 'wb'))
# Read the file later by: X_big = pickle.load(open('path/to/X_big.pkl', "rb"))

NameError: name 'X_big' is not defined

In [5]:
# Keep only rows which belong to cluster 'big'; should be 2,907*123 = 357,561 rows
X_big = X_full[X_full['key'].isin(big_key.astype(str))].reset_index(drop=True)
Y_big = Y_full[Y_full['key'].isin(big_key.astype(str))].reset_index(drop=True)
print(X_big.shape, Y_big.shape)
print(X_big.tail(), Y_big.tail()) # Check that order is still the same

(357561, 108) (357561, 3)
                       key  pid_x         size_x color brand    rrp  \
357556  22872M ( 140-152 )  22872  M ( 140-152 )  blau  Nike  50.73   
357557  22872M ( 140-152 )  22872  M ( 140-152 )  blau  Nike  50.73   
357558  22872M ( 140-152 )  22872  M ( 140-152 )  blau  Nike  50.73   
357559  22872M ( 140-152 )  22872  M ( 140-152 )  blau  Nike  50.73   
357560  22872M ( 140-152 )  22872  M ( 140-152 )  blau  Nike  50.73   

        mainCategory  category  subCategory releaseDate         ...          \
357556             1         7         22.0  2017-10-01         ...           
357557             1         7         22.0  2017-10-01         ...           
357558             1         7         22.0  2017-10-01         ...           
357559             1         7         22.0  2017-10-01         ...           
357560             1         7         22.0  2017-10-01         ...           

       cat_7  cat_10  cat_16  cat_18  cat_24  cat_30  cat_33  cat_36  ca

## Load big cluster

In [6]:
try:
    print(X_big.shape)
except:
    X_big = pickle.load(open('./data/clean/X_big.pkl', 'rb'))
    Y_big = pickle.load(open('./data/clean/Y_big.pkl', 'rb'))

(357561, 108)


## X_big manipulation

In [34]:
x_cols = X_big.columns[12:12+7]
print(x_cols)

Index(['last_1_day_sales', 'last_2_day_sales', 'last_3_day_sales',
       'last_4_day_sales', 'last_5_day_sales', 'last_6_day_sales',
       'last_7_day_sales', 'last_8_day_sales', 'last_9_day_sales',
       'last_10_day_sales', 'last_11_day_sales', 'last_12_day_sales',
       'last_13_day_sales', 'last_14_day_sales', 'last_15_day_sales',
       'last_16_day_sales', 'last_17_day_sales', 'last_18_day_sales',
       'last_19_day_sales', 'last_20_day_sales', 'last_21_day_sales'],
      dtype='object')


In [7]:
# Drop non-numeric columns; networks only take numeric input
# Drop non-numeric columns; networks only take numeric input
keys_dates = pd.DataFrame(X_big['key']).join(X_big['date']) # Store for future lookups


drop_y_cols = ['key', 'date']
Y = Y_big.drop(drop_y_cols, axis=1)

x_cols = X_big.columns[12:12+7]
X = X_big[x_cols]



print(X.shape, Y.shape)

(357561, 7) (357561, 1)


In [8]:
# Convert to numpy to reshape for input
X = X.as_matrix() # Each row has shape (num_vars,)
Y = Y.as_matrix() # Each row has shape (1,)

# 2. Model helpers and parameters

In [9]:
# We need a data generator to create mini-batches of X, Y for us
# For training, X = [0:30], [1:31], ..., [62:92] (total 63 slices per product)
# For training, Y = [29], [30], ..., [91]
# For testing, X = [63:93], [64:94], ..., [93:123] (total 31 slices per product)
# For testing, Y = [92], [93], ..., [122]
# Check whether slices are correct!
class BatchGenerator(object):
    def __init__(self, num_samples, X, Y, batch_size, window_size, num_vars, start_day, end_day):
        self.num_samples = num_samples
        self.X = X
        self.Y = Y
        self.window_size = window_size
        self.batch_size = batch_size
        self.num_vars = num_vars
        self.start_day = start_day # 0 = day 1 (2017-10-01)
        self.end_day = end_day
        self.current_product = 0 # To keep track of which product we are at (of 12,824)
        self.current_day = 0 # To track which day of a product we are in (of 123)

    def generate(self):
        x = np.zeros((self.batch_size, self.window_size, self.num_vars))
        y = np.zeros((self.batch_size, 1)) # Should this be 3d?
        while True:
            for i in range(self.batch_size):
                if (self.current_day+self.start_day+self.window_size) > self.end_day:
                    # Go to next product, first day
                    self.current_product += 1
                    self.current_day = 0
                if self.current_product == self.num_samples:
                    # Go back to first product for next epoch
                    self.current_product = 0
                x[i,:,:] = self.X[self.current_product*123+self.current_day+self.start_day:
                                  self.current_product*123+self.current_day+self.start_day+
                                  self.window_size] # Max = [62:92]
                y[i,:] = self.Y[self.current_product*123+self.current_day+self.start_day+self.window_size-1] # Max = [91]
                self.current_day += 1 
            yield x, y


In [10]:
NUM_SAMPLES = 2907 # Total number of samples (you can use all or just a subset)
WINDOW_SIZE = 30
num_vars = X.shape[1]
train_batch_size = 63 # To make 1 product = 1 batch
# test_batch_size = 31 # To make 1 product = 1 batch

In [11]:
train_start_day = 0
train_end_day = 92 # 2017-10-01 to 2017-12-31
# test_start_day = 92 - WINDOW_SIZE + 1 # So the first window frame ends with 2018-01-01 prediction
# test_end_day = 123 # 2018-01-01 to 2018-01-31

train_data_generator = BatchGenerator(NUM_SAMPLES, X, Y, train_batch_size, WINDOW_SIZE, 
                                      num_vars, train_start_day, train_end_day)
# test_data_generator = BatchGenerator(NUM_SAMPLES, X, Y, test_batch_size, WINDOW_SIZE, 
#                                      num_vars, test_start_day, test_end_day)

In [12]:
# Define customised metric calculations here if we need
# Metric function is similar to a loss function
# except that the results from evaluating a metric are not used when training the model
import keras.backend as backend
def mean_abs_diff(y_true, y_pred):
    return backend.mean(backend.abs(y_true - y_pred))

/Users/d071503/miniconda2/envs/mycondaenv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## At this point, if you already have a model trained previously, 
## you can skip to '5. Prediction' to load the trained model

# 3. Model definition

In [13]:
# Using 5-layered MLP (idea from presentation by Matthias' team)
# Windowing model
# Model predicts only sales unit of last day of a window frame
# Model definition
from keras.models import Sequential
from keras.layers import Dense, Activation, TimeDistributed, LSTM, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

num_epochs = 50 # This is too low; can increase if we push training to the cloud
num_hidden = 64
# Monitor 'loss' instead of 'val_loss', because there is no way to validate actually
callbacks_list = [EarlyStopping(monitor='loss', patience=10), 
                  ModelCheckpoint(filepath='neuralnetworks/experiments/lag7/lag7_best.h5', monitor='loss',save_best_only=True)]
# ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, verbose=1, mode='min', cooldown=0, min_lr=0)

model = Sequential()
model.add(Dense(num_hidden, input_shape=(WINDOW_SIZE, num_vars), activation='relu'))
model.add(Dense(num_hidden, activation='relu'))
model.add(Dense(num_hidden, activation='relu'))
model.add(Dense(num_hidden, activation='relu')) # Add dropout?
model.add(Flatten())
model.add(Dense(1, activation='relu')) # Need kernel_initializer?
model.compile(loss='mean_squared_error', optimizer='adadelta') # To penalise big deviations more
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 30, 64)            512       
_________________________________________________________________
dense_2 (Dense)              (None, 30, 64)            4160      
_________________________________________________________________
dense_3 (Dense)              (None, 30, 64)            4160      
_________________________________________________________________
dense_4 (Dense)              (None, 30, 64)            4160      
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 1921      
Total params: 14,913
Trainable params: 14,913
Non-trainable params: 0
_________________________________________________________________
None


# 4. Model training

In [14]:
# Train LSTM; running this will take a while!
history = model.fit_generator(generator=train_data_generator.generate(),
                              steps_per_epoch=NUM_SAMPLES*(train_end_day-WINDOW_SIZE+1)/train_batch_size,
                              # validation_data=test_data_generator.generate(),
                              # validation_steps=NUM_SAMPLES*(test_end_day-test_start_day-WINDOW_SIZE+1)/test_batch_size, 
                              callbacks=callbacks_list,
                              epochs = num_epochs, verbose=1,
                              shuffle=False)

Epoch 1/50
   1/2907 [..............................] - ETA: 6:53 - loss: 0.2258

  10/2907 [..............................] - ETA: 55s - loss: 1.2595 

  19/2907 [..............................] - ETA: 37s - loss: 0.9011

  28/2907 [..............................] - ETA: 30s - loss: 1.1916

  36/2907 [..............................] - ETA: 27s - loss: 1.3073

...

1315/2907 [============>.................] - ETA: 7s - loss: 6.6422

1326/2907 [============>.................] - ETA: 7s - loss: 6.6003

1337/2907 [============>.................] - ETA: 7s - loss: 6.5527

1349/2907 [============>.................] - ETA: 7s - loss: 6.5134

1360/2907 [=============>................] - ETA: 7s - loss: 6.4624

In [20]:
# Save model and history for future reuse
model.save('neuralnetworks/experiments/lag7/lag7.h5') 
with open('neuralnetworks/experiments/lag7/lag7_history', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

# 5. Prediction

## i) Predict sample

In [42]:
from keras.models import load_model
model = load_model('neuralnetworks/experiments/lag7/lag7.h5') # LOAD THE MODEL YOU WANT HERE

In [43]:
# For future predictions, we have to replace the actual past sales with the predicted sales 
sample_product = '19288L' # Change as required
index = int((keys_dates.index[keys_dates['key'] == sample_product][0])/123) 
predictions = []
product_data = np.zeros(shape=(1, 123, num_vars)) # A copy of the data for replacement
product_data[0, :, :] = product_data[0, 93-WINDOW_SIZE+i:93+i]
product_X = np.zeros(shape=(1, WINDOW_SIZE, num_vars)) # This will be pushed into model
for i in range(31):
    product_X[0, :, :] = product_data[0, 63+i:63+WINDOW_SIZE+i]
    predictions.insert(0, round(model.predict(product_X)[0][0]))
    # Update product data with predictions (we are not supposed to know the actual values)
    if i < num_lags:
        product_data[0, 93+i, 0:i+1] = predictions[0:i+1]
    else:
        product_data[0, 93+i, 0:num_lags] = predictions[0:num_lags]
predictions = predictions[::-1] # We have to reverse the list to get back original order
print("Predicted sales (day1 to last day):")
print(predictions)
print("Actual sales (day1 to last day):")
print(Y[index*123+92:index*123+123].reshape(31,))

ValueError: could not broadcast input array from shape (30,21) into shape (123,21)

Check the sample output above. If predictions look odd, go back to 'Model definition' and 'Model training'. If predictions look ok, proceed to 'Predict all'

## ii) Predict all; takes about 6 minutes

In [44]:
# Helper functions
def pred_soldout_day(pred_sales, stock):
    """
    Helper function to predict soldout day
    """
    result = 15 if stock <= 3 else len(pred_sales) # If stock is too few, middle is a good guess
    for day in range(len(pred_sales)):
        stock -= pred_sales[day]
        if stock <= 0:
            return day+1
    return result

def load_test_set(set_number):
    """
    Helper function to load and prepare test set
    """
    test = pd.read_csv('./data/test/test_{}.csv'.format(str(set_number)), index_col=0, 
                       dtype={'pid': np.int32})
    test['key'] = test['pid'].astype(str) + test['size']
    test = test[test['key'].isin(big_key.astype(str))].reset_index(drop=True) # FOR BIG CLUSTER
    print('Number of products in test data belonging to cluster: ' + str(len(test['key'].unique())))
    
    def extract_soldout_day(row):
        soldout_day = row['sold_out_date'][-2:]
        return int(soldout_day[1]) if soldout_day[0] == 0 else int(soldout_day)
    
    test['soldout_day'] = test.apply(extract_soldout_day, axis=1)
    test['pred_soldout_day'] = np.nan
    return test

In [48]:
# Make predictions for all products and store them
test = load_test_set(0)
all_predictions = pd.DataFrame()

# We only make predictions for the products in the test set
for row in range(test.shape[0]):
    key = str(test.iloc[row]['key'])
    
    # Make predictions for current product
    index = int(keys_dates[keys_dates['key'] == key].index[0]/123) # Get starting row of product
    predictions = []
    product_data = np.zeros(shape=(1, 123, num_vars)) # A copy of the data for replacement
    product_data[0, :, :] = X[index*123:index*123+123]
    
    num_lags=21
    
    product_X = np.zeros(shape=(1, WINDOW_SIZE, num_vars)) # This will be pushed into model
    for i in range(31):
        product_X[0, :, :] = product_data[0, 93-WINDOW_SIZE+i:93+i]
        predictions.insert(0, round(model.predict(product_X)[0][0]))
        # Update product data with predictions (we are not supposed to know the actual values)
        if i < num_lags:
            product_data[0, 93+i, 0:i+1] = predictions[0:i+1]
        else:
            product_data[0, 93+i, 0:num_lags] = predictions[0:num_lags]
    predictions = predictions[::-1] # Reverse the list to get back original order
    all_predictions[key] = predictions

all_predictions.to_csv('neuralnetworks/experiments/lag28_broad/predictions_lag28_broad.csv', index=False)
print("Predictions complete")

Number of products in test data belonging to cluster: 2637


IndexError: index 123 is out of bounds for axis 1 with size 123

# 6. Evaluate on test data; takes about 20 minutes

In [34]:
# Example
prod_pred = all_predictions['12985L'].as_matrix()
print(prod_pred)
print(pred_soldout_day(prod_pred, 50))

[6. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0.]
31


In [35]:
# Go through all test sets
for set_num in range(1):
    test = load_test_set(set_num)
    for row in range(test.shape[0]):
        key = test['key'][row]
        stock = test['stock'][row]
        key_pred = all_predictions[key].as_matrix()
        test['pred_soldout_day'][row] = pred_soldout_day(key_pred, stock)
    test.to_csv('neuralnetworks/experiments/lag28_broad/lag28_broad_result_{}.csv'.format(str(set_num))) # Dump test set result
    print('Test {}: {}'.format(str(set_num), 
                               str(np.sqrt(sum(abs(test['pred_soldout_day'] - test['soldout_day']))))))

Number of products in test data belonging to cluster: 2637


/Users/d071503/miniconda2/envs/mycondaenv/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Test 0: 153.95778642212287


In [26]:
print (num_vars)

28
